# <span style="color:#3CB371;">Mastering Bayesian Optimization: From Theory to Hyperparameter Tuning</span>

<!-- <span style="color:#A9A9A9;"> -->
Bayesian Optimization (BO) is a powerful method for optimizing expensive, black-box, or noisy objective functions. Unlike grid search or random search, BO intelligently chooses the next sampling point using a probabilistic model, reducing the number of evaluations needed.

- This notebook covers:
- 1. Bayesian Optimization Theory
- 2. Pros and Cons
- 3. Applications
- 4. From-scratch Python Implementation
- 5. XGBoost Hyperparameter Tuning with BO
- 6. Comparison with Random and Grid Search
- 7. Limitations
<!-- </span> -->

## <span style="color:#20B2AA;">1. What is Bayesian Optimization?

Bayesian Optimization is a strategy to find the **minimum or maximum of an expensive-to-evaluate function** `f(x)` efficiently. It is particularly useful when:

- Evaluations are costly (time or computation)
- Function is noisy or unknown (black-box)
- Gradient information is unavailable

BO uses two main components:

1. **Surrogate Model**: A probabilistic model (usually a Gaussian Process) approximates `f(x)`.
2. **Acquisition Function**: Determines the next sampling point using the surrogate model to balance **exploration vs. exploitation**.




### <span style="color:#008B8B;">Mathematical Concepts


#### Gaussian Process (GP) Surrogate
A GP defines a distribution over functions:
f(x) ~ GP(m(x), k(x, x'))

Where:
- m(x) = mean function (often 0)
- k(x, x') = covariance/kernel function, e.g., squared exponential:
k(x, x') = exp(-||x-x'||^2 / (2*l^2))

GP gives:
- Mean mu(x) -> predicted function value
- Variance sigma^2(x) -> uncertainty


#### Acquisition Functions
These guide **where to sample next**.


1. **Expected Improvement (EI)**:
EI(x) = E[max(f(x) - f(x^+), 0)]
Where f(x^+) is the current best observed value.


2. **Probability of Improvement (PI)**:
PI(x) = P(f(x) > f(x^+) + xi)


3. **Upper Confidence Bound (UCB)**:
UCB(x) = mu(x) + k * sigma(x)

- k balances exploration (large sigma) vs. exploitation (high mu).

### <span style="color:#008B8B;">Bayesian Optimization Algorithm


1. Initialize: Sample f(x) at a few points.
2. Build GP Surrogate: Fit GP to initial data.
3. Select Next Point: Maximize acquisition function.
4. Evaluate Objective: Compute f(x_next).
5. Update GP: Include new data point.
6. Repeat until convergence criteria are met.

## <span style="color:#20B2AA;">2. Pros and Cons


**Pros:**
- Efficient: fewer evaluations than grid/random search
- Handles noisy/black-box functions
- Provides uncertainty quantification
- Can optimize non-convex, discontinuous functions


**Cons:**
- High computational cost for GP in high dimensions
- Sensitive to choice of surrogate model and acquisition function
- Struggles in very high-dimensional spaces
- Sequential nature (hard to parallelize)


## <span style="color:#20B2AA;">3. Applications

- Hyperparameter tuning (ML models like XGBoost, deep learning)
- Robotics (optimizing gait or control policies)
- Chemical engineering (experiment optimization)
- A/B testing (marketing/product experiments)
- Simulations (scientific experiments with expensive evaluations)

## <span style="color:#20B2AA;">4. Bayesian Optimization from Scratch in Python

In [8]:
import numpy as np
from scipy.stats import norm


# --- Objective Function (Black-box) ---
def f(x):
    return (x - 2)**2 + 3

# --- Gaussian Process Kernel ---
def rbf_kernel(x1, x2, length_scale=1.0, sigma_f=1.0):
    sqdist = np.subtract.outer(x1, x2)**2
    return sigma_f**2 * np.exp(-0.5 / length_scale**2 * sqdist)

# --- Gaussian Process Prediction ---
def gp_predict(X_train, Y_train, X_s, l=1.0, sigma_f=1.0, sigma_y=1e-8):
    K = rbf_kernel(X_train, X_train, l, sigma_f) + sigma_y**2 * np.eye(len(X_train))
    K_s = rbf_kernel(X_train, X_s, l, sigma_f)
    K_ss = rbf_kernel(X_s, X_s, l, sigma_f) + 1e-6 * np.eye(len(X_s))
    K_inv = np.linalg.pinv(K)
    mu_s = K_s.T.dot(K_inv).dot(Y_train)
    cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)
    return mu_s.ravel(), np.diag(cov_s)

# --- Expected Improvement Acquisition ---
def expected_improvement(X_s, X_train, Y_train, xi=0.01):
    mu, sigma = gp_predict(X_train, Y_train, X_s)
    mu_sample_opt = np.min(Y_train)
    with np.errstate(divide='warn'):
        imp = mu_sample_opt - mu - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma==0.0] = 0.0
    return ei

# --- Bayesian Optimization Loop ---
X_train = np.array([0.0, 2.5, 5.0])  # # initial samples
Y_train = f(X_train)


bounds = (0, 5)
n_iter = 10


for i in range(n_iter):
    X_s = np.linspace(bounds[0], bounds[1], 1000)
    ei = expected_improvement(X_s, X_train, Y_train)
    x_next = X_s[np.argmax(ei)]
    y_next = f(x_next)
    X_train = np.append(X_train, x_next)
    Y_train = np.append(Y_train, y_next)
    print(f"Iteration {i+1}: x_next={x_next:.4f}, f(x_next)={y_next:.4f}")


best_idx = np.argmin(Y_train)
print(f"Best x: {X_train[best_idx]:.4f}, f(x)={Y_train[best_idx]:.4f}")

Iteration 1: x_next=1.9069, f(x_next)=3.0087
Iteration 2: x_next=2.0120, f(x_next)=3.0001
Iteration 3: x_next=3.6737, f(x_next)=5.8012
Iteration 4: x_next=0.0000, f(x_next)=7.0000
Iteration 5: x_next=0.0000, f(x_next)=7.0000
Iteration 6: x_next=0.0000, f(x_next)=7.0000
Iteration 7: x_next=0.0000, f(x_next)=7.0000
Iteration 8: x_next=0.0000, f(x_next)=7.0000
Iteration 9: x_next=0.0000, f(x_next)=7.0000
Iteration 10: x_next=0.0000, f(x_next)=7.0000
Best x: 2.0120, f(x)=3.0001


## <span style="color:#20B2AA;">5. XGBoost Hyperparameter Tuning with Bayesian Optimization

In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

# Load Data
data = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.2, random_state=42
)

# Define the hyperparameter search space
space  = [
    Integer(3, 10, name='max_depth'),
    Real(0.01, 0.3, name='learning_rate'),
    Integer(50, 500, name='n_estimators'),
    Real(0.5, 1.0, name='subsample'),
]

# Objective function
@use_named_args(space)
def objective(**params):
    model = xgb.XGBRegressor(
        random_state=42,
        n_jobs=-1,
        **params
    )
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring='neg_root_mean_squared_error')
    return -np.mean(scores)

# Run Bayesian Optimization
res_gp = gp_minimize(objective, space, n_calls=20, random_state=42)

print("Best score: %.4f" % res_gp.fun)
print("Best parameters:")
for name, val in zip([s.name for s in space], res_gp.x):
    print(f"  {name}: {val}")


Best score: 0.4674
Best parameters:
  max_depth: 9
  learning_rate: 0.0631960890611875
  n_estimators: 401
  subsample: 0.7984250789732436


## <span style="color:#20B2AA;">6. Advantages over Random/Grid Search


| Feature | Grid Search | Random Search | Bayesian Optimization |
|---------------------------|------------|---------------|---------------------|
| Evaluations Required | High | Medium | Low |
| Handles Expensive Func | ❌ | ❌ | ✅ |
| Exploits Past Knowledge | ❌ | ❌ | ✅ |
| Can Optimize Noisy Funcs | ❌ | ❌ | ✅ |


BO learns from previous trials, focusing on promising regions—unlike random or grid search which wastes evaluations.

## <span style="color:#20B2AA;">7. Limitations


- **Scalability**: GP surrogate scales poorly in high dimensions
- **Computational Overhead**: Inverting kernel matrices is expensive
- **Sequential**: Hard to parallelize without modifications
- **Hyperparameter Sensitivity**: Choice of kernel and acquisition function affects performance

## <span style="color:#20B2AA;">✅ Conclusion


Bayesian Optimization provides an **efficient, probabilistic framework** for optimizing black-box functions and tuning ML hyperparameters. While it has limitations in high dimensions, for expensive or noisy functions, BO often outperforms random and grid search.